In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
warnings.filterwarnings('ignore')

In [3]:
link = r"C:\Users\rApiiNel\Legalmatch\Finance and Data Analysis Team - Sales Ops\Python\26 Salesforce Blank Practice Area Fix\v1\raw"

In [4]:
file = glob.glob(link + '\*3*')

In [5]:
df = pd.read_csv(file[0], encoding='latin-1')

#### Checking accounts that has comma in the practice area

##### Test

In [6]:
comma = df.dropna(subset = ['Practice Areas'])

In [7]:
comma_test = comma[comma['Salesforce Account Id'] == '0014y00002aVQd7AAG']

In [8]:
def comma_cleaner(df, column):
    df['Cleaned ' + column] = df.apply(lambda x: ';'.join(list(set(x[column].replace(',',';').split(';')))), axis= 1)
    df['Length ' + column] = df.apply(lambda x: len(x['Cleaned '+column].split(';')), axis = 1)

In [9]:
comma_cleaner(comma_test, 'Practice Areas')

In [10]:
comma_test.to_csv('comma_test.csv', index= False)

##### Actual

In [11]:
comma_cleaner(comma, 'Practice Areas')

In [12]:
comma = comma[comma['Practice Areas'].str.contains(',')].drop_duplicates(subset= ['Salesforce Account Id'])
comma['Cleaned Practice Areas'] = comma.apply(lambda x: x['Cleaned Practice Areas'].strip(), axis = 1)
comma[['Salesforce Account Id', 'Cleaned Practice Areas']].to_csv('comma.csv', index= False)

#### Contact with Comma

In [13]:
comma_contact = df.dropna(subset = ['Practice Areas.1']).copy()

In [14]:
comma_cleaner(comma_contact, 'Practice Areas.1')

In [15]:
comma_contact = comma_contact[comma_contact['Length Practice Areas.1'] > 1]

In [16]:
comma_contact = comma_contact[comma_contact['Practice Areas.1'].str.contains(',')]
comma_contact = comma_contact[['Salesforce Contact Id', 'Cleaned Practice Areas.1']]

In [17]:
comma_contact.to_csv('comma_contact.csv', index= False)


#### Account and Contact Comparisson

In [18]:
contact_df = df.dropna(subset = ['Practice Areas.1'])

In [19]:
comma_cleaner(contact_df, 'Practice Areas.1')

In [20]:
contact_df = contact_df.groupby('Salesforce Account Id').agg({'Cleaned Practice Areas.1':lambda x: ';'.join([x.strip() for x in list(set(';'.join(x).split(';')))])})

In [21]:
account_df = df.dropna(subset = ['Practice Areas'])
comma_cleaner(account_df, 'Practice Areas')

In [22]:
account_df = account_df.groupby('Salesforce Account Id').agg({'Cleaned Practice Areas':lambda x: ';'.join([x.strip() for x in list(set(';'.join(x).split(';')))])})

In [23]:
from collections import Counter as counter
counter(contact_df.index.isin(account_df.index))

Counter({True: 330792, False: 6})

In [24]:
comp_df = account_df.merge(contact_df, left_index=True, right_index=True)

In [25]:
comp_df['Difference'] = comp_df.progress_apply(lambda x: set(x['Cleaned Practice Areas.1'].split(';')) - set(x['Cleaned Practice Areas'].split(';')), axis= 1)

  0%|          | 0/330792 [00:00<?, ?it/s]

In [26]:
comp_df['Length Difference'] = comp_df.progress_apply(lambda x: len(x['Difference']), axis = 1)

  0%|          | 0/330792 [00:00<?, ?it/s]

In [31]:
comp_df['To be uploaded in Account Practice Areas'] = comp_df.progress_apply(lambda x: '; '.join(x['Difference']), axis= 1)

  0%|          | 0/330792 [00:00<?, ?it/s]

In [27]:
comp_df[comp_df['Length Difference'] > 0]

,Cleaned Practice Areas,Cleaned Practice Areas.1,Difference,Length Difference
Salesforce Account Id,,,,
0010e00001JRDMbAAP,Government;Government and Administrative;Plann...,Government;Zoning;Planning and Land Use;Govern...,{Law and Government/Procurement},1
0010e00001JRDNHAA5,Government Administration and Regulation;State...,Government;Administrative;Corporate;Insurance;...,{Governance and Rule of Law},1
0010e00001JRDOdAAP,Commercial Litigation;Environmental and Land U...,Environmental and Land Use;Labor and Employmen...,{Environment/Natural Resources},1
0010e00001JRDQPAA5,Environmental;Real Estate;Criminal;Civil Right...,Environmental;Real Estate;Disability Law | Leg...,{Disability Law | Legal Malpractice | Medical ...,1
0010e00001JRDQfAAP,Business/ Commercial;General;Legal Research an...,Business/ Commercial;General;Legal Research an...,{Debtor-Creditor},1
...,...,...,...,...
0016000000ogh6kAAA,Real Estate;Employment;Product Liability;Insur...,Administrative;Education;Real Estate;Employmen...,"{Administrative, Education}",2
0016000000ojmGoAAI,City/County/Local Government;Civil Litigation;...,Constitutional;City/County/Local Government;Ci...,{Law and Government/Procurement},1
0016000000pG2eqAAC,Cyberspace;General Practice;Business Transacti...,General Practice;Business Transactional;Securi...,{Business litigation},1


# Conclusion

1. There are occurrence of practice area being displayed in salesforce that are not found in the drop down.
    - This can possible affect plan - 3133 matching
1. Check initial data on practice areas not included in the picklist
    - How many records are affected?
    - what is the possible cause?
        - Check possible charactiristics
    - how many specific practice areas are need to be corrected?